#  HyperGAT
hypergraph attention networks
paper: https://aclanthology.org/2020.emnlp-main.399.pdf

github: https://github.com/kaize0409/HyperGAT_TextClassification/tree/main

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nm6104054/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
dataset = pd.read_csv('dataset/20ng/20ng_over512.csv', names=['label','text'])
dataa = "20ng"
len(dataset['label']*0.5)

1137

In [3]:

# from scipy.sparse import coo_array
# row  = np.array([0, 0, 1, 3, 1, 0, 0])
# col  = np.array([0, 2, 1, 3, 1, 0, 0])
# data = np.array([1, 1, 1, 1, 1, 1, 1])
# coo = coo_array((data, (row, col)), shape=(4, 5))
# # Duplicate indices are maintained until implicitly or explicitly summed
# np.max(coo.data)
# coo.toarray()
# # array([[3, 0, 1, 0],
#        [0, 2, 0, 0],
#        [0, 0, 0, 0],
#        [0, 0, 0, 1]])

In [4]:
# train_model(model, train_data)


## pass 

In [3]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
# from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import re
import collections
from collections import Counter
import numpy as np
from multiprocessing import Process, Queue
import pandas as pd
import os
import random
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.utils import *



def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def clean_str_simple_version(string, dataset):

    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()


def show_statisctic(clean_docs):
    min_len = 10000
    aver_len = 0
    max_len = 0 
    num_sentence = sum([len(i) for i in clean_docs])
    ave_num_sentence = num_sentence*1.0/len(clean_docs)

    for doc in clean_docs:
        for sentence in doc:
            temp = sentence
            aver_len = aver_len + len(temp)

            if len(temp) < min_len:
                min_len = len(temp)
            if len(temp) > max_len:
                max_len = len(temp)

    aver_len = 1.0 * aver_len / num_sentence

    print('min_len_of_sentence : ' + str(min_len))
    print('max_len_of_sentence : ' + str(max_len))
    print('min_num_of_sentence : ' + str(min([len(i) for i in clean_docs])))
    print('max_num_of_sentence : ' + str(max([len(i) for i in clean_docs])))
    print('average_len_of_sentence: ' + str(aver_len))
    print('average_num_of_sentence: ' + str(ave_num_sentence))
    print('Total_num_of_sentence : ' + str(num_sentence))

    return max([len(i) for i in clean_docs])


def clean_document(doc_sentence_list, dataset):

    stop_words = stopwords.words('english')
    stop_words = set(stop_words)
    stemmer = WordNetLemmatizer()

    word_freq = Counter()

    for doc_sentences in doc_sentence_list:
        for sentence in doc_sentences:
            temp = word_tokenize(clean_str(sentence))
            temp = ' '.join([stemmer.lemmatize(word) for word in temp])

            words = temp.split()
            for word in words:
                word_freq[word] += 1

    highbar = word_freq.most_common(10)[-1][1]
    clean_docs = []
    for doc_sentences in doc_sentence_list:
        clean_doc = []
        count_num = 0
        for sentence in doc_sentences:
            temp = word_tokenize(clean_str(sentence))
            temp = ' '.join([stemmer.lemmatize(word) for word in temp])

            words = temp.split()
            doc_words = []
            for word in words:
                # if dataset == 'mr':
                #     if not word in stop_words:
                #         doc_words.append(word)
                # elif (word not in stop_words) and (word_freq[word] >= 5) and (word_freq[word] < highbar):
                #     doc_words.append(word)
                    
                if (word not in stop_words) and (word_freq[word] >= 5) and (word_freq[word] < highbar):
                    doc_words.append(word)

            clean_doc.append(doc_words)
            count_num += len(doc_words)

            # if dataset == '20ng' and count_num > 2000:
            #     break
            
        clean_docs.append(clean_doc)

    return clean_docs

def split_validation(train_set, valid_portion, SEED):
    np.random.seed(SEED)

    train_set_x = [i for i,j in train_set]
    train_set_y = [j for i,j in train_set]

    if valid_portion == 0.0:
        return (train_set_x, train_set_y)

    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)

import nltk
nltk.download('punkt')

class Data():
    def __init__(self, data, max_num_sentence, keywords_dic, num_categories, LDA = False):
        inputs = data[0]
        self.inputs = np.asarray(inputs) 
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.keywords = keywords_dic
        self.LDA = LDA
        self.num_categories = num_categories

    def generate_batch(self, batch_size, shuffle = False):
        if shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.targets = self.targets[shuffled_arg]

        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, iList):
        inputs, targets = self.inputs[iList], self.targets[iList]
        items, n_node, HT, alias_inputs, node_masks, node_dic = [], [], [], [], [], []

        for u_input in inputs:
            temp_s = []
            for s in u_input:
                temp_s += s
            
            temp_l = list(set(temp_s))    
            temp_dic = {temp_l[i]: i for i in range(len(temp_l))}        
            n_node.append(temp_l)
            alias_inputs.append([temp_dic[i] for i in temp_s])
            node_dic.append(temp_dic)


        max_n_node = np.max([len(i) for i in n_node])


        num_edge = [len(i) for i in inputs]

        if self.LDA:
            num_edge = [i + self.num_categories  for i in num_edge]

        max_n_edge = max(num_edge)

        max_se_len = max([len(i) for i in alias_inputs])

        for idx in range(len(inputs)):
            u_input = inputs[idx]
            node = n_node[idx]
            items.append(node + (max_n_node - len(node)) * [0])


            rows = []
            cols = []
            vals = []

            
            for s in range(len(u_input)):
                for i in np.arange(len(u_input[s])):
                    if u_input[s][i] == 0:
                        continue

                    rows.append(node_dic[idx][u_input[s][i]])
                    cols.append(s)
                    vals.append(1.0)

            if len(cols) == 0:
                s = 0
            else:
                s = max(cols) + 1

            if self.LDA:
                for i in node:
                    if i in self.keywords:
                        temp = self.keywords[i]
                                                
                        rows += [node_dic[idx][i]]*len(temp)
                        cols += [topic + s for topic in temp]
                        vals += [1.0]*len(temp)
                                

            u_H = sp.coo_matrix((vals, (rows, cols)), shape=(max_n_node, max_n_edge))
            HT.append(np.asarray(u_H.T.todense()))
            
            alias_inputs[idx] = [j for j in range(max_n_node)]
            node_masks.append([1 for j in node] + (max_n_node - len(node)) * [0])


        return alias_inputs, HT, items, targets, node_masks


[nltk_data] Downloading package punkt to /home/nm6104054/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import pickle
from sklearn.utils import class_weight
from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
# from utils import clean_str, clean_str_simple_version, show_statisctic, clean_document
import sys
from nltk import tokenize
import collections
from collections import Counter
import random
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np 


def display_topics(model, feature_names, no_top_words):
    keywords_dic = {}
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        klist = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(" ".join(klist))
        for k in klist:
            if not k in keywords_dic:
                keywords_dic[k] = []
            keywords_dic[k].append(topic_idx)
    return keywords_dic

def Generate_LDA(data,topics):
    dataset = pd.read_csv('dataset/20ng/20ng_over512.csv', names=['label','text'])
    dataset['text']
    doc_content_list = []
    doc_sentence_list = []
    # f = open('dataa/' + dataset + '_corpus.txt', 'rb')
    for line in dataset['text']:
        doc_content_list.append(line)
        doc_sentence_list.append(tokenize.sent_tokenize(clean_str_simple_version(doc_content_list[-1], dataset)))
    # f.close()

    # Remove the rare words
    doc_content_list = clean_document(doc_sentence_list, dataset)

    doc_list = []
    for doc in doc_content_list:
        temp = ''
        for sen in doc:
            temp += (' '.join(sen) + ' ')
        doc_list.append(temp)

    vectorizer = CountVectorizer(stop_words='english',max_df=0.98)
    vector = vectorizer.fit_transform(doc_list)
    feature_names = vectorizer.get_feature_names_out()
    lda = LatentDirichletAllocation(n_components=topics,learning_method='online',learning_offset=50.,random_state=0).fit(vector)

    keywords_dic = display_topics(lda, feature_names, 10)
    print(len(keywords_dic))
    print(keywords_dic)

    pickle.dump(keywords_dic, open('data/' + data+ '_LDA.p', "wb" ) )


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--dataset', default='20ng', help='dataset name: 20ng/R8/R52/ohsumed/mr')
    # parser.add_argument('--topn', type=int, default=10, help='top n keywords')
    # parser.add_argument('--topics', type=int, help='number of topics')
    topics = 20
    topn = 10 
    # args = parser.parse_args()
    # print(args)
    data = "20ng"
    Generate_LDA(data,topics)


Topic 0:
drive db scsi disk bit window card hard controller mov
Topic 1:
nazi hitler german limbaugh jew germany page nlns liberal chancellor
Topic 2:
archiving facto drake concert edu warrant wa com people os2
Topic 3:
wa ha people data conclusion know absolute argument religion remote
Topic 4:
armenian wa turkish people greek turk turkey said armenia muslim
Topic 5:
space nasa mission satellite earth orbit gov shuttle probe launch
Topic 6:
god jesus christian wa say bible people doe believe life
Topic 7:
25 10 55 16 11 14 12 13 15 17
Topic 8:
church father catholic pope holy son spirit maria schism bishop
Topic 9:
key encryption chip clipper bit security enforcement escrow algorithm device
Topic 10:
00 edu com 50 1st appears 10 25 copy 40
Topic 11:
wa ha people edu dont like time think com year
Topic 12:
team player game ca hockey season play gm playoff fan
Topic 13:
plane den point p3 p2 radius p1 sphere p13 p23
Topic 14:
tulkarm torture shabak detention interrogation intercon amand

In [5]:

from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
# from utils import clean_str, show_statisctic, clean_document, clean_str_simple_version
import collections
from collections import Counter
import random
import numpy as np
import pickle
import json
from nltk import tokenize
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder


def read_file(dataname, LDA=True):

    doc_content_list = []
    doc_sentence_list = []
#     f = open('data/' + dataset + '_corpus.txt', 'rb')

#     for line in f.readlines():
#         doc_content_list.append(line.strip().decode('latin1'))
#         doc_sentence_list.append(tokenize.sent_tokenize(clean_str_simple_version(doc_content_list[-1], dataset)))
#     f.close()

    dataset = pd.read_csv('dataset/20ng/20ng_over512.csv', names=['label','text'])
    dataset['text']
    doc_content_list = []
    doc_sentence_list = []
    # f = open('dataa/' + dataset + '_corpus.txt', 'rb')
    for line in dataset['text']:
        doc_content_list.append(line)
        doc_sentence_list.append(tokenize.sent_tokenize(clean_str_simple_version(doc_content_list[-1], dataname)))


    doc_content_list = clean_document(doc_sentence_list, dataset)

    max_num_sentence = show_statisctic(doc_content_list)

    doc_train_list_original = []
    doc_test_list_original = []
    labels_dic = {}
    label_count = Counter()

    # 改掉
    i = 0
    LE = LabelEncoder()
    a=int(len(dataset['label']))*0.5
    dataset['label'] = LE.fit_transform(dataset['label'])
    for index, temp in enumerate(dataset['label']):
        if index < a : 
            doc_train_list_original.append((doc_content_list[i],temp))
        else :
            doc_test_list_original.append((doc_content_list[i],temp))
        if not temp in labels_dic:
            labels_dic[temp] = len(labels_dic)
        i += 1    
#     f = open('data/' + dataset + '_labels.txt', 'r')
#     lines = f.readlines()
#     for line in lines:
#         temp = line.strip().split("\t")
#         if temp[1].find('test') != -1:
#             doc_test_list_original.append((doc_content_list[i],temp[2]))
#         elif temp[1].find('train') != -1:
#             doc_train_list_original.append((doc_content_list[i],temp[2]))
        # if not temp in labels_dic:
        #     labels_dic[temp[2]] = len(labels_dic)
#         label_count[temp[2]] += 1
#        

#     f.close()
#     print(label_count)



    word_freq = Counter()
    word_set = set()
    for doc_words in doc_content_list:
        for words in doc_words:
            for word in words:
                word_set.add(word)
                word_freq[word] += 1

    vocab = list(word_set)
    vocab_size = len(vocab)

    vocab_dic = {}
    for i in word_set:
        vocab_dic[i] = len(vocab_dic) + 1

    print('Total_number_of_words: ' + str(len(vocab)))
    print('Total_number_of_categories: ' + str(len(labels_dic)))

    doc_train_list = []
    doc_test_list = []

    for doc,label in doc_train_list_original:
        temp_doc = []
        for sentence in doc:
            temp = []
            for word in sentence:
                temp.append(vocab_dic[word])
            temp_doc.append(temp)
        doc_train_list.append((temp_doc,labels_dic[label]))

    for doc,label in doc_test_list_original:
        temp_doc = []
        for sentence in doc:
            temp = []
            for word in sentence:
                temp.append(vocab_dic[word])
            temp_doc.append(temp)
        doc_test_list.append((temp_doc,labels_dic[label]))

    keywords_dic = {}
    if LDA:
        keywords_dic_original = pickle.load(open('data/' + dataname + '_LDA.p', "rb" ))

        for i in keywords_dic_original:
            if i in vocab_dic:
                keywords_dic[vocab_dic[i]] = keywords_dic_original[i]

    train_set_y = [j for i,j in doc_train_list]

    class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(train_set_y),y=train_set_y)

    return doc_content_list, doc_train_list, doc_test_list, vocab_dic, labels_dic, max_num_sentence, keywords_dic, class_weights


def loadGloveModel(gloveFile, vocab_dic, matrix_len):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    gloveModel = {}
    glove_embedding_dimension = 0
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        glove_embedding_dimension = len(splitLine[1:])
        embedding = np.array([float(val) for val in splitLine[1:]])
        gloveModel[word] = embedding

    words_found = 0
    weights_matrix = np.zeros((matrix_len, glove_embedding_dimension))
    weights_matrix[0] = np.zeros((glove_embedding_dimension, ))

    for word in vocab_dic:
        if word in gloveModel:
            weights_matrix[vocab_dic[word]] = gloveModel[word]
            words_found += 1
        else:
            weights_matrix[vocab_dic[word]] = gloveModel['the']

    print("Total ", len(vocab_dic), " words")
    print("Done.",words_found," words loaded from", gloveFile)

    return weights_matrix


In [6]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter


class HyperGraphAttentionLayerSparse(nn.Module):

    def __init__(self, in_features, out_features, dropout, alpha, transfer, concat=True, bias=False):
        super(HyperGraphAttentionLayerSparse, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.transfer = transfer

        if self.transfer:
            self.weight = Parameter(torch.Tensor(self.in_features, self.out_features))
        else:
            self.register_parameter('weight', None)

        self.weight2 = Parameter(torch.Tensor(self.in_features, self.out_features))
        self.weight3 = Parameter(torch.Tensor(self.out_features, self.out_features))

        if bias:
            self.bias = Parameter(torch.Tensor(self.out_features))
        else:
            self.register_parameter('bias', None)

        self.word_context = nn.Embedding(1, self.out_features)
      
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))   
        self.a2 = nn.Parameter(torch.zeros(size=(2*out_features, 1)))        
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.out_features)
        if self.weight is not None:
            self.weight.data.uniform_(-stdv, stdv)
        self.weight2.data.uniform_(-stdv, stdv)
        self.weight3.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
        
        nn.init.uniform_(self.a.data, -stdv, stdv)
        nn.init.uniform_(self.a2.data, -stdv, stdv)
        nn.init.uniform_(self.word_context.weight.data, -stdv, stdv)


    def forward(self, x, adj):
        x_4att = x.matmul(self.weight2)

        if self.transfer:
            x = x.matmul(self.weight)
            if self.bias is not None:
                x = x + self.bias        

        N1 = adj.shape[1] #number of edge
        N2 = adj.shape[2] #number of node

        pair = adj.nonzero().t()        

        get = lambda i: x_4att[i][adj[i].nonzero().t()[1]]
        x1 = torch.cat([get(i) for i in torch.arange(x.shape[0]).long()])


        q1 = self.word_context.weight[0:].view(1, -1).repeat(x1.shape[0],1).view(x1.shape[0], self.out_features)
        
        pair_h = torch.cat((q1, x1), dim=-1)
        pair_e = self.leakyrelu(torch.matmul(pair_h, self.a).squeeze()).t()
        assert not torch.isnan(pair_e).any()
        pair_e = F.dropout(pair_e, self.dropout, training=self.training)

        e = torch.sparse_coo_tensor(pair, pair_e, torch.Size([x.shape[0], N1, N2])).to_dense()

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)


        attention_edge = F.softmax(attention, dim=2)

        edge = torch.matmul(attention_edge, x)
        
        edge = F.dropout(edge, self.dropout, training=self.training)

        edge_4att = edge.matmul(self.weight3)

        get = lambda i: edge_4att[i][adj[i].nonzero().t()[0]]
        y1 = torch.cat([get(i) for i in torch.arange(x.shape[0]).long()])

        get = lambda i: x_4att[i][adj[i].nonzero().t()[1]]
        q1 = torch.cat([get(i) for i in torch.arange(x.shape[0]).long()])

        pair_h = torch.cat((q1, y1), dim=-1)
        pair_e = self.leakyrelu(torch.matmul(pair_h, self.a2).squeeze()).t()
        assert not torch.isnan(pair_e).any()
        pair_e = F.dropout(pair_e, self.dropout, training=self.training)

        e = torch.sparse_coo_tensor(pair, pair_e, torch.Size([x.shape[0], N1, N2])).to_dense()

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)

        attention_node = F.softmax(attention.transpose(1,2), dim=2)

        node = torch.matmul(attention_node, edge)


        if self.concat:
            node = F.elu(node)

        return node

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [14]:
import datetime
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F
from sklearn import metrics
from tqdm import tqdm


def trans_to_cuda(variable):
    if torch.cuda.is_available():
        return variable.cuda()
    else:
        return variable


def trans_to_cpu(variable):
    if torch.cuda.is_available():
        return variable.cpu()
    else:
        return variable


class HGNN_ATT(nn.Module):
    def __init__(self, input_size, n_hid, output_size, dropout=0.3):
        super(HGNN_ATT, self).__init__()
        self.dropout = dropout
        self.gat1 = HyperGraphAttentionLayerSparse(input_size, n_hid, dropout=self.dropout, alpha=0.2, transfer = False, concat=True)
        self.gat2 = HyperGraphAttentionLayerSparse(n_hid, output_size, dropout=self.dropout, alpha=0.2, transfer = True, concat=False)
        
    def forward(self, x, H):   
        x = self.gat1(x, H)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gat2(x, H)

        return x


class DocumentGraph(Module):
    def __init__(self, pre_trained_weight, class_weights, n_node, n_categories):
        super(DocumentGraph, self).__init__()
        epoch = 10; lr = 0.005 ; dropout = 0.3; lr_dc = 0.1 ; lr_dc_step = 3; l2 = 1e-6 ; valid_portion =0.1; rand = 1234; normalization = True
        self.hidden_size =100
        self.n_node = n_node
        self.n_categories = n_categories
        self.batch_size = 8
        self.dropout = 0.3   
        self.initial_feature = 300
        self.normalization = True
        # self.dataset = opt.dataset


        self.embedding = nn.Embedding(self.n_node+1, self.initial_feature, padding_idx=0)
        self.layer_normH = nn.LayerNorm(self.hidden_size, eps=1e-6)
        if self.normalization:
            self.layer_normC = nn.LayerNorm(self.n_categories, eps=1e-6)

        self.prediction_transform = nn.Linear(self.hidden_size, self.n_categories, bias=True)  

        self.reset_parameters()

        # if opt.dataset == 'mr':
        #     pre_trained_weight = torch.FloatTensor(pre_trained_weight)
        #     self.embedding = nn.Embedding.from_pretrained(pre_trained_weight, freeze = False, padding_idx = 0)

        self.hgnn = HGNN_ATT(self.initial_feature, self.initial_feature, self.hidden_size, dropout = self.dropout)

        self.class_weights = class_weights
        self.loss_function = nn.CrossEntropyLoss(weight = trans_to_cuda(torch.Tensor(self.class_weights).float()))
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=lr_dc_step, gamma=lr_dc)
        
        

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
        

    def compute_scores(self, inputs, node_masks):
        
        hidden = inputs * node_masks.view(node_masks.shape[0], -1, 1).float()
        b = torch.sum(hidden * node_masks.view(node_masks.shape[0], -1, 1).float(),-2)/torch.sum(node_masks,-1).repeat(hidden.shape[2],1).transpose(0,1)          
        b = self.layer_normH(b)           
        b = self.prediction_transform(b)

        pred = b

        if self.normalization:
            pred = self.layer_normC(b)
        
        return pred

    def forward(self, inputs, HT):
        
        hidden = self.embedding(inputs)
        nodes = self.hgnn(hidden, HT)
        return nodes


def forward(model, alias_inputs, HT, items, targets, node_masks):

    alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
    items = trans_to_cuda(torch.Tensor(items).long())
    HT = trans_to_cuda(torch.Tensor(HT).float())
    node_masks = trans_to_cuda(torch.Tensor(node_masks).float())
    node = model(items, HT)
    get = lambda i: node[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])
    return targets, model.compute_scores(seq_hidden, node_masks)


def train_model(model, train_data):
    lr = 0.001 ; dropout = 0.3; lr_dc = 0.1 ; lr_dc_step = 3; l2 = 1e-6 ; valid_portion =0.1; rand = 1234; normalization = True; use_LDA =True

    model.scheduler.step()
    print('start training: \n')
    model.train()
    total_loss = 0.0
    slices = train_data.generate_batch(batchSize, True)
    for step in tqdm(range(len(slices)), total=len(slices), ncols=70, leave=False, unit='b'):
        i = slices[step]
        alias_inputs, HT, items, targets, node_masks = train_data.get_slice(i)
        model.optimizer.zero_grad()
        targets, scores = forward(model, alias_inputs, HT, items, targets, node_masks)    
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets)
        loss.backward()
        model.optimizer.step()
        total_loss += loss

    print('\tLoss:\t%.4f' % (total_loss))



def test_model(model, test_data, verbose=True):
    model.eval()

    test_pred = []
    test_labels = []
    slices = test_data.generate_batch(10, False)
    for step in tqdm(range(len(slices)), total=len(slices), ncols=70, leave=False, unit='b'):
        i = slices[step]
        alias_inputs, HT, items, targets, node_masks = test_data.get_slice(i)
        targets, scores = forward(model, alias_inputs, HT, items, targets, node_masks)
        pre_indices = scores.topk(1)[1]
        test_labels += list(targets)
        test_pred += list(trans_to_cpu(pre_indices).detach().numpy())

    details = metrics.classification_report(test_labels, test_pred, digits=4)
    acc = metrics.accuracy_score(test_labels, test_pred)

    if verbose:
        print("Test Precision, Recall and F1-Score...")
        print(metrics.classification_report(test_labels, test_pred, digits=4))
        print("Macro average Test Precision, Recall and F1-Score...")
        print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
        print("Micro average Test Precision, Recall and F1-Score...")
        print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))

    return details,acc

In [15]:
# train_model(model, train_data)

## Data 

In [16]:
import argparse
import pickle
import time
import numpy as np
# from utils import split_validation, Data
# from preprocess import *
# from model import *
from sklearn.utils import class_weight
import random
import warnings
import os

warnings.filterwarnings('ignore') 


dataname = "20ng"
batchSize = 8;hiddenSize = 100;initialFeatureSize = 300
epochs = 10; lr = 0.001 ; dropout = 0.3; lr_dc = 0.1 ; lr_dc_step = 3; l2 = 1e-6 ; valid_portion =0.1; rand = 1234; normalization = True
use_LDA = False



SEED =rand
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)



# def main():
doc_content_list, doc_train_list, doc_test_list, vocab_dic, labels_dic, max_num_sentence, keywords_dic, class_weights = read_file(dataname, True)

pre_trained_weight = []
gloveFile = 'glove.6B.300d.txt'  

pre_trained_weight = loadGloveModel(gloveFile, vocab_dic, len(vocab_dic)+1)

train_data, valid_data = split_validation(doc_train_list, valid_portion, SEED)
test_data = split_validation(doc_test_list, 0.0, SEED)
train_data, valid_data

num_categories = len(labels_dic)

train_data = Data(train_data, max_num_sentence, keywords_dic, num_categories, use_LDA)
valid_data = Data(valid_data, max_num_sentence, keywords_dic, num_categories, use_LDA)
test_data = Data(test_data, max_num_sentence,  keywords_dic, num_categories, use_LDA)



min_len_of_sentence : 0
max_len_of_sentence : 7265
min_num_of_sentence : 1
max_num_of_sentence : 405
average_len_of_sentence: 11.473086453744493
average_num_of_sentence: 51.109938434476696
Total_num_of_sentence : 58112
Total_number_of_words: 13428
Total_number_of_categories: 20
Loading Glove Model
Total  13428  words
Done. 12580  words loaded from glove.6B.300d.txt


In [17]:

model = trans_to_cuda(DocumentGraph(pre_trained_weight, class_weights, len(vocab_dic)+1, len(labels_dic)))

for epoch in range(20):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    train_model(model, train_data)

    valid_detail, valid_acc = test_model(model, valid_data, False)
    detail, acc = test_model(model, test_data, False)
    print('Validation Accuracy:\t%.4f, Test Accuracy:\t%.4f'% (valid_acc,acc))

-------------------------------------------------------
epoch:  0
start training: 



	Loss:	181.4580


Validation Accuracy:	0.5965, Test Accuracy:	0.6813
-------------------------------------------------------
epoch:  1
start training: 



	Loss:	136.0115


Validation Accuracy:	0.8421, Test Accuracy:	0.8380
-------------------------------------------------------
epoch:  2
start training: 



...

Validation Accuracy:	0.8596, Test Accuracy:	0.8697
-------------------------------------------------------
epoch:  11
start training: 



	Loss:	87.2526


Validation Accuracy:	0.8596, Test Accuracy:	0.8697
-------------------------------------------------------
epoch:  12
start training: 



	Loss:	88.5505


KeyboardInterrupt: 